
<H1> FedAVG


Connect the Notebook with GoogleDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0, '/content/drive/MyDrive/FL2022/FedAVG')

Mounted at /content/drive


Import Libraries

In [2]:
from options import args_parser
from update import LocalUpdate, test_inference, client_update, server_aggregate, test
from utils import get_dataset, average_weights, weighted_average_weights, exp_details, plot_client_distribution, get_n_params
from modelRN50 import ResNet50
import torch
torch.cuda.empty_cache()
import torch.optim as optim
import numpy as np
import pandas as pd
from tqdm import tqdm # progress bar
import copy
from torch.utils.tensorboard import SummaryWriter
logger = SummaryWriter('../logs')

Set the parameters for the training 

In [3]:
sys.argv=['',
          '--iid=1',  #0 -> NONiid, 1 -> iid
          '--num_users=100',
          '--lr=0.0001',
          '--local_ep=1',
          '--epochs=300',
          '--optimizer=adam',
          '--norm=batch_norm',
          '--local_bs=10',
          '--dataset=cifar',
          '--loss=CrossEntropyLoss',
          '--gpu=/device:GPU:0']
args=args_parser()
num_selected = int( args.num_users * args.frac )
baseline_num = 100 # number of baseline images to be saved on the global server
                   # for retraining of the client's model before aggregation
unbalanced = False # if True the clients will contain different number of classes
verbose = False

Set the device on cuda

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if verbose:
 print(f"Device available: {device}")

###Build the Model ResNet-50

In [5]:
global_model = ResNet50(args.norm)
global_model.to(device)
global_model.train()
if verbose:
  print(global_model)
  print(f"Number of Parameters: {get_n_params(global_model)}")

global_weights = global_model.state_dict()
client_models = [ global_model for _ in range(num_selected)]

# Synchronizing the clients with the global model 
for model in client_models:
    model.load_state_dict(global_model.state_dict()) 
# Optimizer selection
if args.optimizer == 'sgd':
    opt = [optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=args.wd) for model in client_models]
elif args.optimizer == 'adam':
    opt = [optim.Adam(model.parameters(), lr=args.lr,  weight_decay=args.wd) for model in client_models]

####Dataset split:

In [6]:
train_dataset, test_dataset, user_groups = get_dataset(args=args, unbalanced=unbalanced,)
if verbose:
  plot_client_distribution(user_groups, train_dataset)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


#Training

In [7]:
# training
train_loss, train_accuracy = [], []
test_acc_list, test_loss_list = [], []

for epoch in range(1, args.epochs+1):
    local_weights = []
    local_losses = []
    print(f'Epoch: {epoch} \n')
    model.train()
    m = max(int(args.frac * args.num_users), 1) # num of users, at least 1
    idxs_users = np.random.choice(range(args.num_users), m, replace=False) 

    for idx in idxs_users: 
        local_model = LocalUpdate(args=args, dataset=train_dataset, idxs=user_groups[idx],logger=logger)
        w, loss = local_model.update_weights(model=copy.deepcopy(model), # pass the global model to the clients
                                             global_round=epoch)
        print('| Client : {} | Average Loss: {:.4f} '.format(idx, loss))
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))

    # compute global weights (average of local weights)
    if unbalanced:
        global_weights = weighted_average_weights(local_weights, user_groups, idxs_users)
    else:
        global_weights = average_weights(local_weights)

    # update weights of the global model
    model.load_state_dict(global_weights)

    # compute average loss
    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    model.eval()

    # calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    for client in range(args.num_users): 
        local_model = LocalUpdate(args=args, dataset=train_dataset, idxs=user_groups[client],logger=logger)
        acc, loss = local_model.inference(model=model)
        list_acc.append(acc)
        list_loss.append(loss)

    train_accuracy.append(sum(list_acc)/len(list_acc))
    print(f'\nAverage training statistics (global epoch) : {epoch}')
    print(f'|---- Trainig Loss : {np.mean(np.array(train_loss))}')
    print('|---- Training Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))
    test_loss, test_acc = test(args, model,global_model, test_dataset)
    test_acc_list.append(test_acc)
    test_loss_list.append(test_loss)
    print('%d-th round' %epoch)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))

Epoch: 1 



/content/drive/MyDrive/FL2022/FedAVG/update.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)


| Global Round : 1 | Local Epoch : 1 | Loss: 2.466485 | Accuracy: 9.78%
| Global Round : 1 |	Loss avg: 2.466485 
| Client : 55 | Average Loss: 2.4665 
| Global Round : 1 | Local Epoch : 1 | Loss: 2.489828 | Accuracy: 10.22%
| Global Round : 1 |	Loss avg: 2.489828 
| Client : 98 | Average Loss: 2.4898 
| Global Round : 1 | Local Epoch : 1 | Loss: 2.414156 | Accuracy: 12.89%
| Global Round : 1 |	Loss avg: 2.414156 
| Client : 39 | Average Loss: 2.4142 
| Global Round : 1 | Local Epoch : 1 | Loss: 2.393677 | Accuracy: 11.78%
| Global Round : 1 |	Loss avg: 2.393677 
| Client : 95 | Average Loss: 2.3937 
| Global Round : 1 | Local Epoch : 1 | Loss: 2.378166 | Accuracy: 11.56%
| Global Round : 1 |	Loss avg: 2.378166 
| Client : 23 | Average Loss: 2.3782 
| Global Round : 1 | Local Epoch : 1 | Loss: 2.383941 | Accuracy: 13.11%
| Global Round : 1 |	Loss avg: 2.383941 
| Client : 36 | Average Loss: 2.3839 
| Global Round : 1 | Local Epoch : 1 | Loss: 2.431785 | Accuracy: 12.67%
| Global Round :

KeyboardInterrupt: ignored

#Storing the Results

In [ ]:
dict_ = {'train_acc' : train_accuracy, 'train_loss' : train_loss}
df = pd.DataFrame(dict_) 
iid = ['iid' if args.iid else 'nonIID']
unb = ['unbalanced' if unbalanced and not args.iid else 'balanced' ]
bs = args.local_bs 
filename = f"fedAVG_{iid}_{unb}_{args.norm}{bs}_{args.epochs}_lr_{args.lr}_optimizer_{args.optimizer}"
df = pd.DataFrame(dict_) 
df.to_csv('/content/drive/MyDrive/FL2022/FedAVG/Results/'+filename+'.csv', encoding='utf-8')

Showing the Results

In [ ]:
print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
